# NB-AK4: Long-term annual total fluxes %changes for early, mid and late 21st century periods from historical period

## module loading...

In [ ]:
%matplotlib inline  
import os
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from timeit import default_timer as timer

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)

mpl.rcParams['xtick.labelsize']='small'

In [ ]:
from cmip5_oconus import print_date
print_date()

from cmip5_oconus.data_catalog import load_monthly_historical_hydro_datasets, resample_data
from cmip5_oconus.data_catalog import load_monthly_cmip5_hydro_datasets
from cmip5_oconus.plot import custom_div_cmap, MidpointNormalize, add_ylabel
from cmip5_oconus.utils import calc_change

In [ ]:
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster(processes=9, threads=4, memory="108GB",
#                  walltime='00:40:00')
#cluster.scale(jobs=3)

In [ ]:
#from dask.distributed import Client
#client = Client(cluster)

In [ ]:
#client

-------------------------
## Setup 

In [ ]:
figSave = True

In [ ]:
gcms            = ["ACCESS1-3","CanESM2","CCSM4","CSIRO-Mk3-6-0","GFDL-ESM2M","HadGEM2-ES","inmcm4","MIROC5","MPI-ESM-MR","MRI-CGCM3"]
hydro_flux_vars = ['PRCP', 'EVAP', 'total_runoff']
period_hist     = slice('1970-01-01', '1999-12-31')
period_futr     = {'early' : slice('2010-01-01', '2039-12-31'),
                   'middle': slice('2040-01-01', '2069-12-31'),
                   'late'  : slice('2070-01-01', '2099-12-31')}

## Load the data

In [ ]:
xr.set_options(file_cache_maxsize=12000)
histo_data = load_monthly_historical_hydro_datasets(models=gcms, variables=hydro_flux_vars, region='AK', parallel=False)
rcp45_data = load_monthly_cmip5_hydro_datasets(scen='rcp45', models=gcms, variables=hydro_flux_vars, region='AK', parallel=False)
rcp85_data = load_monthly_cmip5_hydro_datasets(scen='rcp85', models=gcms, variables=hydro_flux_vars, region='AK', parallel=False)

## Compute 30-yr annual mean for 4 periods - historical, early, mid and late

In [ ]:
histo_hydro_means = resample_data(histo_data['gcm'].sel(time=period_hist), region='AK', freq='AS').mean('time').compute()

rcp45_hydro_means = {}
rcp85_hydro_means = {}
for key, futr_period in period_futr.items():
    rcp45_hydro_means[key] = resample_data(rcp45_data.sel(time=futr_period), region='AK', freq='AS').mean('time').compute()
    rcp85_hydro_means[key] = resample_data(rcp85_data.sel(time=futr_period), region='AK', freq='AS').mean('time').compute()

## Compute difference in 30-yr annual mean between historical and future

In [ ]:
rcp45_hydro_30yr_diff = {}
rcp85_hydro_30yr_diff = {}
diff_rcp_hydro_30yr_change ={}
for key, futr_period in period_futr.items():
    rcp45_hydro_30yr_diff[key] = calc_change(histo_hydro_means, rcp45_hydro_means[key], pct=True)
    rcp85_hydro_30yr_diff[key] = calc_change(histo_hydro_means, rcp85_hydro_means[key], pct=True)
    
    diff_rcp_hydro_30yr_change[key] =rcp85_hydro_30yr_diff[key] - rcp45_hydro_30yr_diff[key]

## Plotting

In [ ]:
# create colormap
# ---------------

# create a colormap that consists of
# - 1/5 : custom colormap, ranging from white to the first color of the colormap
# - 4/5 : existing colormap

# set upper part: 4 * 256/4 entries
upper = mpl.cm.Spectral_r(np.arange(256))

# set lower part: 1 * 256/4 entries
# - initialize all entries to 1 to make sure that the alpha channel (4th column) is 1
lower = np.ones((int(256/4),4))
# - modify the first three columns (RGB):
#   range linearly between white (1,1,1) and the first color of the upper colormap
for i in range(3):
    lower[:,i] = np.linspace(0.9, upper[0,i], lower.shape[0])

# combine parts of colormap
cmap = np.vstack(( lower, upper ))

# convert to matplotlib colormap
cmap = mpl.colors.ListedColormap(cmap, name='myColorMap', N=cmap.shape[0])

In [ ]:
# create colormap
# ---------------

from matplotlib.colors import LinearSegmentedColormap

# create a colormap that consists of
# - 1/5 : custom colormap, ranging from white to the first color of the colormap
# - 4/5 : existing colormap

# set upper part: 4 * 256/4 entries
upper = mpl.cm.Spectral_r(np.arange(256))

# set lower part: 1 * 256/4 entries
# - initialize all entries to 1 to make sure that the alpha channel (4th column) is 1
lower = np.ones((int(256/4),4))
# - modify the first three columns (RGB):
#   range linearly between white (1,1,1) and the first color of the upper colormap
for i in range(3):
    lower[:,i] = np.linspace(0.9, upper[0,i], lower.shape[0])

# combine parts of colormap
cmap = np.vstack(( lower, upper ))

# convert to matplotlib colormap
cmap = mpl.colors.ListedColormap(cmap, name='myColorMap', N=cmap.shape[0])

cmap_change_RO = LinearSegmentedColormap.from_list('custom2', 
                                             [(0,    'xkcd:red'),
                                              (0.4,  'xkcd:light grey'),
                                              (1,    'xkcd:blue')], N=256)

------------
 - Change in 30yr mean annual total fluxes between historical and three future periods in 21st century (early, mid, and late) 

In [ ]:
cm = {'history'   : cmap,
      'change'    : custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'change_RO' : cmap_change_RO}

norm = {'PRCP': MidpointNormalize(vmin=-100, vmax=100, midpoint=0),
        'EVAP': MidpointNormalize(vmin=-100, vmax=100, midpoint=0),
        'ROFF': MidpointNormalize(vmin=-100, vmax=150, midpoint=0)}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'change': {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean':   {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'change': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['change']}
               }

kwargs = {'change-PRCP':         {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'],    'extend':'max',  **style_kwargs['change']},
          'change-EVAP':         {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'],    'extend':'both', **style_kwargs['change']},
          'change-total_runoff': {'levels': 26, 'vmin':-100, 'vmax':150, 'cmap':cm['change_RO'], 'extend':'max',  **style_kwargs['change']},
          'PRCP':                {'levels': 31, 'vmin':0, 'vmax':3000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']},
          'EVAP':                {'levels': 17, 'vmin':0, 'vmax':400,  'cmap':cm['history'], 'extend':'both', **style_kwargs['mean']},
          'total_runoff':        {'levels': 31, 'vmin':0, 'vmax':3000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']}}

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=4, ncols=3, sharex=True, sharey=True, figsize=(8.125,11.25))
plt.subplots_adjust(left=0.045, bottom=0.0125, right=0.985, top=0.975, hspace=0.075, wspace=0.075)

for col, var in enumerate(['PRCP','EVAP','total_runoff']):
    histo_hydro_means.mean(dim='gcm')[var].plot.pcolormesh(ax=axes[0, col], **kwargs[var])
    rcp85_hydro_30yr_diff['early'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[1, col],  **kwargs[f'change-{var}'])
    rcp85_hydro_30yr_diff['middle'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[2, col], **kwargs[f'change-{var}'])
    rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[3, col],   **kwargs[f'change-{var}'])
    
    add_ylabel(axes[0, 0], 'historical [mm/yr]', fontsize=11)
    add_ylabel(axes[1, 0], 'changes 2010-2039 [%]', fontsize=11)
    add_ylabel(axes[2, 0], 'changes 2040-2069 [%]', fontsize=11)
    add_ylabel(axes[3, 0], 'changes 2070-2099 [%]', fontsize=11)
    axes[0, col].set_title(var_title[var], fontsize=12)
    
#plt.tight_layout()
if figSave:
    plt.savefig(f'NB-AK4_Fig1_annual_mean_pchange_rcp85.png', dpi=300)

------------
 - The same as above except for two future periods in 21st century (early, mid, and late) 

In [ ]:
cm = {'history'   : cmap,
      'change'    : custom_div_cmap(numcolors=25, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'change_RO' : cmap_change_RO}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'change': {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean':   {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'change': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['change']}
               }

kwargs = {'change-PRCP':         {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'],    'extend':'max',  **style_kwargs['change']},
          'change-EVAP':         {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'],    'extend':'both', **style_kwargs['change']},
          'change-total_runoff': {'levels': 26, 'vmin':-100, 'vmax':150, 'cmap':cm['change_RO'], 'extend':'max',  **style_kwargs['change']},
          'PRCP':                {'levels': 31, 'vmin':0, 'vmax':3000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']},
          'EVAP':                {'levels': 17, 'vmin':0, 'vmax':400,  'cmap':cm['history'], 'extend':'both', **style_kwargs['mean']},
          'total_runoff':        {'levels': 31, 'vmin':0, 'vmax':3000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']}}

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=3, ncols=3, sharex=True, sharey=True, figsize=(8.125,8.75))
plt.subplots_adjust(left=0.045, bottom=0.0125, right=0.985, top=0.975, hspace=0.075, wspace=0.075)

for col, var in enumerate(['PRCP','EVAP','total_runoff']):
    histo_hydro_means.mean(dim='gcm')[var].plot.pcolormesh(ax=axes[0, col], **kwargs[var])
    rcp85_hydro_30yr_diff['middle'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[1, col], **kwargs[f'change-{var}'])
    rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[2, col],   **kwargs[f'change-{var}'])
    
    add_ylabel(axes[0, 0], 'historical [mm/yr]', fontsize=11)
    add_ylabel(axes[1, 0], 'changes 2040-2069 [%]', fontsize=11)
    add_ylabel(axes[2, 0], 'changes 2070-2099 [%]', fontsize=11)
    axes[0, col].set_title(var_title[var], fontsize=12)
    
if figSave:
    plt.savefig(f'NB-AK4_Fig2_annual_mean_pchange_mid_late_rcp85.png', dpi=300)

------------
 - Difference between two RCPs in change in 30yr mean annual total for late 21st century

In [ ]:
cm = {'change' : custom_div_cmap(numcolors=25, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue')}

style_kwargs = {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.995, pad=0.03)}
kwargs       = {'change-rcp': {'vmin':-100, 'vmax':100,  'cmap':cm['change'], 'extend':'both', **style_kwargs},
               }

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(8.125,3))
plt.subplots_adjust(left=0.040, bottom=0.0125, right=0.990, top=0.900, hspace=0.060, wspace=0.060)

for col, var in enumerate(['PRCP','EVAP','total_runoff']):
    diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[col], **kwargs['change-rcp'])
    
    add_ylabel(axes[0], 'RCP85 - RCP45[%]', fontsize=11)
    axes[col].set_title(var_title[var], fontsize=12)
    
if figSave:
    plt.savefig(f'NB-AK4_Fig3_annual_mean_pchange_late_diff_rcp.png', dpi=300)

--------
- Climate changes for individual GCMs for late 21st century are plotted below

--------------------
### Precipitation

#### rcp45

In [ ]:
var="PRCP"

cm = {'change': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std'   : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'], 'extend':'max',  **style_kwargs},
          'std':    {'levels': 11,  'vmin':0,   'vmax':50,  'cmap':cm['std'],    'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
#plt.tight_layout()
fig.suptitle('%Change in 30yr mean annual precipitation - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig4_annual_mean_pchange_late_{var}_rcp45.png', dpi=300)

In [ ]:
var="PRCP"

cm = {'change': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std'   : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'], 'extend':'max',  **style_kwargs},
          'std':    {'levels': 11,  'vmin':0,   'vmax':50,  'cmap':cm['std'],    'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('%Change in 30yr mean annual precipitation - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig5_annual_mean_pchange_late_{var}_rcp85.png', dpi=300)

#### rcp85 - rcp45

In [ ]:
var="PRCP"

cm = {'mean': custom_div_cmap(numcolors=25, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=24, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}
kwargs = {'mean': {'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 7,  'vmin':0,   'vmax':60, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
fig.suptitle('RCP85 vs RCP 45: %Change in 30yr mean annual precipitation - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig6_annual_mean_pchange_late_{var}_rcp_diff.png', dpi=300)

#### rcp85

--------------------
### total runoff

#### rcp45

In [ ]:
var="total_runoff"

cm = {'change': cmap_change_RO,
      'std' :   custom_div_cmap(numcolors=21, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 26, 'vmin':-100, 'vmax':150, 'cmap':cm['change'],'extend':'max', **style_kwargs},
          'std':    {'levels': 10, 'vmin':0,    'vmax':100, 'cmap':cm['std'],   'extend':'max', **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
#plt.tight_layout()
fig.suptitle('%Change in 30yr mean annual total runoff - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig7_annual_mean_pchange_late_{var}_rcp45.png', dpi=300)

#### rcp85

In [ ]:
var="total_runoff"

cm = {'change': cmap_change_RO,
      'std' :   custom_div_cmap(numcolors=21, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 26, 'vmin':-100, 'vmax':150, 'cmap':cm['change'],'extend':'max', **style_kwargs},
          'std':    {'levels': 10, 'vmin':0,    'vmax':100, 'cmap':cm['std'],   'extend':'max', **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('%Change in 30yr mean annual total runoff - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig8_annual_mean_pchange_late_{var}_rcp85.png', dpi=300)

#### rcp85 - rcp45

In [ ]:
var="total_runoff"

cm = {'mean': custom_div_cmap(numcolors=25, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=24, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}
kwargs = {'mean': {'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 7,  'vmin':0,   'vmax':60, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
fig.suptitle('RCP85 vs RCP 45: %Change in 30yr mean annual total runoff - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig9_annual_mean_pchange_late_{var}_rcp_diff.png', dpi=300)

-------------------------
### Evapotranspiration

#### rcp45

In [ ]:
var="EVAP"

cm = {'change': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std'   : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'], 'extend':'both',  **style_kwargs},
          'std':    {'levels': 6,  'vmin':0,   'vmax':50,  'cmap':cm['std'],    'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('%Change in 30yr mean annual evapotranspiration - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig10_annual_mean_pchange_late_{var}_rcp45.png', dpi=300)

#### rcp85

In [ ]:
var="EVAP"

cm = {'change': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std'   : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'change': {'levels': 21, 'vmin':-100, 'vmax':100, 'cmap':cm['change'], 'extend':'both',  **style_kwargs},
          'std':    {'levels': 6,  'vmin':0,   'vmax':50,  'cmap':cm['std'],    'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['change'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('%Change in 30yr mean annual evapotransipiration - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig11_annual_mean_pchange_late_{var}_rcp85.png', dpi=300)

#### RCP85 - RCP45

In [ ]:
var="EVAP"

cm = {'mean': custom_div_cmap(numcolors=25, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=24, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}
kwargs = {'mean': {'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 9,  'vmin':0,   'vmax':40, 'cmap':cm['std'],  'extend':'max', **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('RCP85 vs RCP 45: %Change in 30yr mean annual evapotransipiration - late 21st century', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-AK4_Fig12_annual_mean_pchange_late_{var}_rcp_diff.png', dpi=300)